## Generating Context  form a Looker Explore

Use this colab to generate context  + examples from an existing Looker Explore. Make sure to connect to your Looker Instance first, Run Setup, Connect to Looker and Python Functions. Finally Run the Gradio App to use the interface.

DISCLAIMER: THIS IS SAMPLE CODE ONLY, IT IS NOT INTENDED FOR USE IN PRODUCTION ENVIRONMENTS

In [ ]:
# @title Setup
!pip install looker_sdk
!pip install pandas
!pip install -q gradio

In [ ]:
#@title Connect to Looker
import looker_sdk
import requests
import pandas as pd
import urllib3
import os
import json
import yaml
import string
import random
import time
import gradio as gr
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

os.environ['LOOKERSDK_BASE_URL'] = 'https://ceworkshops.looker.com' # @param {type:"string"}
os.environ['LOOKERSDK_CLIENT_ID'] = 'CLIENT_ID' #@param {type:"string"}
os.environ['LOOKERSDK_CLIENT_SECRET'] = 'CLIENT_SECRET' #@param {type:"string"}
os.environ['LOOKERSDK_VERIFY_SSL']= 'False'
os.environ['LOOKERSDK_API_VERSION']= '4.0'
os.environ["LOOKERSDK_TIMEOUT"] = "120"

sdk = looker_sdk.init40()

In [ ]:
#@title Python Functions : can be used independepntly.
def get_dict(model_name, explore_name):
    # API Call to pull in metadata about fields in a particular explore
    explore = sdk.lookml_model_explore(
        lookml_model_name=model_name,
        explore_name=explore_name,
        fields="id, name, description, fields, label",
    )

    measures = []
    dimensions = []

    # Iterate through the dimensions and measures in the explore
    if explore.fields and explore.fields.dimensions:
        for dimension in explore.fields.dimensions:
          if not dimension.hidden:
            def_dimension = {
                "name": dimension.name,
                "description": dimension.description,
            }
            dimensions.append(def_dimension)

    if explore.fields and explore.fields.measures:
        for measure in explore.fields.measures:
          if not measure.hidden:
            def_measure = {
              "name": measure.name,
              "description": measure.description,
            }
            measures.append(def_measure)

    gr_dimensions = ""
    gr_measures = ""

    for d in dimensions:
      dim_print = d['name']
      desc_print = d['description']
      gr_dimensions = gr_dimensions +f"{dim_print}:{desc_print}\n"


    for m in measures:
      m_print = m['name']
      m_desc = m['description']
      gr_measures = gr_measures+ f"{m_print}:{m_desc}\n"

    return f"Dimensions : \n {gr_dimensions} \n Measures : \n {gr_measures}"

# Generate Standrized Examples : modify your dimension by the time dimension in your explore.
def gen_std_examples(time_dim):
  # this functions takes a standrized list of examples and replace 'events.event' by the user choosen time_dim (without the timeframe ! view.dimension eg order_items.)
  response = requests.get('https://raw.githubusercontent.com/Kriz182/explore-assistant/main/stnd_examples.txt')
  response = response.text.replace("events.event", time_dim)
  return response



def generate_examples_dashboard(dashboard_id, explore_name, model_name):
  ### Enforcing Explore Name and Model Name in order to filter out the given examples within a dashboard.
  # Body as an template
  body = {
  "view": "dashboard",
  "fields": [
    "dashboard_element.title",
    "query.id"
  ],
  "filters": {
    "dashboard.moved_to_trash": "No",
    "dashboard.id": "4608",
    "query.view": "\"employee_data\"",
    "query.model": "thelook"
  },
  "model": "system__activity"
  }

  ### Updating values of the template
  body['filters']['dashboard.id'] = str(dashboard_id)
  body['filters']['query.view'] = explore_name
  body['filters']['query.model'] = model_name

  ### Running the query against system_activity of the instance to return the used dashboard title and query.id
  dashboard_queries = sdk.run_inline_query(body=body, result_format='json')

  ### Preparing the Output.
  output_results = []
  dashboard_queries = json.loads(dashboard_queries)
  formatted_results = ""

  ### Iterating through the differents dashboards queries
  for item in dashboard_queries:
        title = item['dashboard_element.title']
        query_id = str(item['query.id'])

      ## Running Looker API Call to Get the Query.URL
        url_response = sdk.query(query_id=query_id)
        url_reponse = url_response.url

      # Formatting the URL to bused in the examples part of Explore Assisant.
      # Important : we're removing the &vis= to make it simple as Looker introduce many parameters that are not useful for the training.
      # However, you can additional visualisation parameters or only extract the type of the visualisation from the vis parameters
      # You need also to review the filters, as sometimes they are being put by default in the dashboards.
        start_index = url_reponse.find('fields=')
        end_index = url_reponse.find('&vis=')
        looker_query_url = url_reponse[start_index:end_index]

      ### Formatinng the result as input and output to be used in our context. You can choose differents ways of fromatting depending on where you want to use the examples.
        output_results.append(f"input: {title}\noutput: {looker_query_url}\n")
  formatted_results = "\n".join(output_results)
  #print(formatted_results)
  return formatted_results

## Launch Gradio App
Run the code blow to Launch the Gradio App in the Colab or Externally.
Alternatively, you can run the **functions above independently**.

In [ ]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
    """
    # Explore Assistant : sandbox environement, demo config.

    """)

    with gr.Tab(label="1. Generate Context"):
        with gr.Row():
            with gr.Column(scale=1):
                ui_model = gr.Textbox(label="Model Name")
                ui_explore = gr.Textbox(label="Explore Name")
                ui_submit_button = gr.Button(value="Generate Context")
            with gr.Column(scale=5):
                ui_context = gr.Textbox(label="Dictionary", show_copy_button=True)
                ui_submit_button.click(fn=get_dict, inputs=[ui_model, ui_explore], outputs=ui_context)

    with gr.Tab(label="2. Standrized Examples"):
        with gr.Row():
            with gr.Column(scale=1):
                ui_time_dim = gr.Textbox(label="Insert default time dimension eg `order_items.created` without the time frame", value ="view.dimension")
                ui_button_ex = ui_submit_button = gr.Button(value="Generate Examples")
            with gr.Column(scale=5):
                ui_button_ex.click(fn=gen_std_examples, inputs=ui_time_dim, outputs=gr.Textbox(label="Standrized Examples", show_copy_button=True))
    with gr.Tab(label="3. Dashboard Examples"):
        with gr.Row():
            with gr.Column(scale=1):
                ui_dash = gr.Textbox(label="Dashboard ID")
                ui_model = gr.Textbox(label="Model Name")
                ui_explore = gr.Textbox(label="Explore Name")
                ui_ex_submit_button = gr.Button(value="Generate Examples")
            with gr.Column(scale=5):
                ui_dashboard = gr.Textbox(label="Dashboard Examples", show_copy_button=True)
                ui_ex_submit_button.click(fn=generate_examples_dashboard, inputs=[ui_dash, ui_explore, ui_model], outputs=ui_dashboard)
    with gr.Tab(label="4. Recent Queries Examples"):
        1

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://18f66088fc8f0b0922.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://18f66088fc8f0b0922.gradio.live
